Logger wrapper personalization

We will see in this notebook how to personalize the logger wrapper defined by sinergym

In [1]:
import gym
import numpy as np
import sinergym
from sinergym.utils.wrappers import (LoggerWrapper, MultiObsWrapper,
                                     NormalizeObservation)
from sinergym.utils.constants import RANGES_5ZONE

/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


First we need to change the CSV logger to modify the values written into the file on the function create_row_contents

In [2]:
from sinergym.utils.logger import CSVLogger
from typing import Any, Dict, Optional, Sequence, Tuple, Union, List

class CustomCSVLogger(CSVLogger):

    def __init__(
            self,
            monitor_header: str,
            progress_header: str,
            log_progress_file: str,
            log_file: Optional[str] = None,
            flag: bool = True):
        super(CustomCSVLogger, self).__init__(monitor_header,progress_header,log_progress_file,log_file,flag)
        self.last_10_steps_reward = [0]*10

    def _create_row_content(
            self,
            obs: List[Any],
            action: Union[int, np.ndarray, List[Any]],
            reward: Optional[float],
            done: bool,
            info: Optional[Dict[str, Any]]) -> List:

        if reward is not None:
            self.last_10_steps_reward.pop(0)
            self.last_10_steps_reward.append(reward)

        if info is None:  # In a reset
            return [0] + list(obs) + list(action) + \
                [0, reward, np.mean(self.last_10_steps_reward), None, None, None, done]
        else:
            return [
                info['timestep']] + list(obs) + list(action) + [
                info['time_elapsed'],
                reward,
                np.mean(self.last_10_steps_reward),
                info['total_power_no_units'],
                info['comfort_penalty'],
                info['abs_comfort'],
                done]